# Titanic - Machine Learning from Disaster

<img src="https://storage.googleapis.com/kaggle-competitions/kaggle/3136/logos/header.png" width=900>

- Vamos utilizar os [dados disponíveis no Kaggle](https://www.kaggle.com/competitions/titanic)
    - É um dataset de **competição**
    - O resultado é avaliado através da **acurácia**:
        - _"Sua pontuação é a porcentagem de passageiros que você prevê corretamente. Isso é conhecido como acurácia."_

### Importando novamente as bases e fazendo o tratamento dos dados
- Vamos apenas replicar o que fizemos no **[primeiro](https://github.com/lucaslealx/Titanic/blob/main/Parte1.ipynb)** e no **[segundo](https://github.com/lucaslealx/Titanic/blob/main/Parte2.ipynb)** arquivo dessa análise (para visualizar os arquivos completos, visite esse link)

In [ ]:
# Importando o pandas
import pandas as pd

In [ ]:
# Visualizando a base de treino
treino = pd.read_csv('train.csv')
treino.head(3)

In [ ]:
# Visualizando a base de teste
teste = pd.read_csv('test.csv')
teste.head(3)

- **Fazendo o mesmo tratamento inicial que fizemos nas aulas anteriores**

In [ ]:
# Eliminando as colunas com elevada cardinalidade
treino = treino.drop(['Name','Ticket','Cabin'],axis=1)
teste = teste.drop(['Name','Ticket','Cabin'],axis=1)

In [ ]:
# Usando a média para substituir valores nulos na coluna de idade
treino.loc[treino.Age.isnull(),'Age'] = treino.Age.mean()
teste.loc[teste.Age.isnull(),'Age'] = teste.Age.mean()

In [ ]:
# Tratando a coluna Embarked da base de treino usando a moda 
treino.loc[treino.Embarked.isnull(),'Embarked'] = treino.Embarked.mode()[0]

In [ ]:
# E também a coluna Fare da base de teste usando a média
teste.loc[teste.Fare.isnull(),'Fare'] = teste.Fare.mean()

In [ ]:
# Usando uma lambda function para tratar a coluna "Sex"
treino['MaleCheck'] = treino.Sex.apply(lambda x: 1 if x == 'male' else 0)
teste['MaleCheck'] = teste.Sex.apply(lambda x: 1 if x == 'male' else 0)

- E vamos usar o **[OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)** para tratar a coluna Embarked

In [ ]:
# Criando o encoder e fazendo o fit com os dados
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(handle_unknown='ignore',dtype='int32')
ohe = ohe.fit(treino[['Embarked']])

In [ ]:
# Atualizando a base de treino com os novos valores
ohe_df = pd.DataFrame(ohe.transform(treino[['Embarked']]).toarray(),columns=ohe.get_feature_names_out())
treino = pd.concat([treino,ohe_df],axis=1)

In [ ]:
# e a base de teste
ohe_df = pd.DataFrame(ohe.transform(teste[['Embarked']]).toarray(),columns=ohe.get_feature_names_out())
teste = pd.concat([teste,ohe_df],axis=1)

In [ ]:
# Podemos então apagar essas duas colunas que já tratamos
treino = treino.drop(['Sex','Embarked'],axis=1)
teste = teste.drop(['Sex','Embarked'],axis=1)

- Visualizando a base resultante

In [ ]:
# Visualizando a base de treino
treino.head(3)

- Podemos já [utilizar essa base no modelo](#modelo) para efeitos de comparação com as melhorias que vamos fazer abaixo

<a id='ajustes'></a>
### Vamos fazer alguns ajustes nos dados para melhorar o resultado do nosso modelo
- [Podemos começar analisando a escala dos dados](#escala)
- [Depois podemos entender as colunas SibSp e Parch](#familia)
- [Selecionando as melhores variáveis do modelo](#f_selection)
    - [Entendendo a relação entre **Pclass e Fare**](#PclassFare)
    - [Entendendo sobre as **colunas de embarque**](#Embarque)
    - [Entendendo agora as **colunas de família**](#Familia)

<a id='escala'></a>
#### Analisando a escala dos dados
- As colunas **"Age" e "Fare"** estão em uma **escala muito diferente das outras**, podemos começar fazendo um ajuste nessas escalas
    - Vamos usar os **[ajustes de escala do próprio Scikit-Learn](https://scikit-learn.org/stable/modules/preprocessing.html#standardization-or-mean-removal-and-variance-scaling)**
    - Como essas colunas possuem outliers, podemos começar tentando utilizar o **[RobustScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler)**

In [ ]:
# Visualizando a escala desses dados
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(15,5))

ax.boxplot(treino.iloc[:,1:11])
ax.set_xticks(range(1,treino.iloc[:,1:11].shape[1]+1),treino.iloc[:,1:11].columns)

plt.show()

In [ ]:
# Importando o RobustScaler

In [ ]:
# Criando o scaler

In [ ]:
# Fazendo o transformação dos dados

In [ ]:
# Fazendo o mesmo para a base de teste

In [ ]:
# Visualizando novamente os dados
fig, ax = plt.subplots(figsize=(15,5))

ax.boxplot(treino.iloc[:,1:11])
ax.set_xticks(range(1,treino.iloc[:,1:11].shape[1]+1),treino.iloc[:,1:11].columns)

plt.show()

- Podemos já [utilizar essa base no modelo](#modelo) para verificar o resultado

<a id='familia'></a>
#### Entendendo as colunas SibSp e Parch
- Agora podemos entender as colunas **SibSp** e **Parch**
    - **SibSp**: Nº de irmãos/cônjuges a bordo do Titanic
    - **Parch**: Nº de pais/filhos a bordo do Titanic
- Podemos verificar quantas pessoas sobreviveram em cada um dos valores dessa coluna

In [ ]:
# Verificando a sobrevivência para SibSp

In [ ]:
# Verificando a sobrevivência para Parch

In [ ]:
# Visualizando essas informações graficamente
fig, ax = plt.subplots(ncols=2,nrows=2,figsize=(10,6))

ax[0,0].plot(SibSp_df.SibSp, SibSp_df.sobrev)
ax[0,0].plot(SibSp_df.SibSp, SibSp_df.total)
ax[0,0].set_title('sobreviventes por nº de irmãos/cônjuges', fontsize=10)

labels1 = ax[0,1].bar(SibSp_df.SibSp, SibSp_df.tx_sobrev)
ax[0,1].bar_label(labels1,fmt="%.02f")
ax[0,1].set(ylim=(0,0.6))
ax[0,1].set_title('taxa de sobrevivência por nº de irmãos/cônjuges', fontsize=10)

ax[1,0].plot(Parch_df.Parch, Parch_df.sobrev)
ax[1,0].plot(Parch_df.Parch, Parch_df.total)
ax[1,0].set_title('sobreviventes por nº de pais/filhos', fontsize=10)

labels2 = ax[1,1].bar(Parch_df.Parch, Parch_df.tx_sobrev)
ax[1,1].bar_label(labels2,fmt="%.02f")
ax[1,1].set(ylim=(0,0.7))
ax[1,1].set_title('taxa de sobrevivência por nº de pais/filhos', fontsize=10)

plt.subplots_adjust(hspace=0.5)

plt.show()

- **Passageiros viajando sozinho tiveram um menor percentual de sobrevivência**
    - Podemos criar uma nova coluna identificando se o passageiro está viajando sozinho

In [ ]:
# Criando uma função para verificar se os dois valores são vazios

In [ ]:
# Aplicando essa função na base de treino

In [ ]:
# Verificando os valores nessa coluna

In [ ]:
# Fazendo o mesmo para a base de teste

- Podemos também **criar uma nova coluna** sendo o **total de familiares a bordo**

In [ ]:
# Criando para a base de treino

In [ ]:
# E para a base de teste

In [ ]:
# Verificando a sobrevivência para Familiares

In [ ]:
# Visualizando a informação de familiares
fig, ax = plt.subplots(ncols=2,figsize=(10,3))

ax[0].plot(Familiares_df.Familiares, Familiares_df.sobrev)
ax[0].plot(Familiares_df.Familiares, Familiares_df.total)
ax[0].set_title('sobreviventes por nº de familiares', fontsize=10)

labels1 = ax[1].bar(Familiares_df.Familiares, Familiares_df.tx_sobrev)
ax[1].bar_label(labels1,fmt="%.02f")
ax[1].set(ylim=(0,0.8))
ax[1].set_title('taxa de sobrevivência por nº de familiares', fontsize=10)

plt.show()

- Podemos já [utilizar essa base no modelo](#modelo) para verificar o resultado

<a id="f_selection"></a>
#### Selecionando os melhores recursos
- Primeiramente precisamos analisar qual a **correlação entre as variáveis** e verificar se não existem variáveis que estão basicamente "dizendo a mesma coisa"

In [ ]:
# Visualizando a base

In [ ]:
# Analisando a correlação entre as variáveis

In [ ]:
# Tornando a correlação mais visual
import seaborn as sns

fig, ax = plt.subplots(figsize=(10,5))

sns.heatmap(treino.corr(), annot=True, fmt=".2f")

plt.show()

<a id='PclassFare'></a>
- <font color='blue'> Entendendo a relação entre **Pclass e Fare** </font>

In [ ]:
# Visualizando a relação entre essas variáveis
fig, ax = plt.subplots(figsize=(10,3))

sns.boxplot(data=treino,x='Pclass',y='Fare',hue='Survived')
ax.set_title('Pclass x Fare', fontsize=10)

plt.show()

In [ ]:
# Verificando a taxa de sobrevivência em cada uma das classes

In [ ]:
# Entendendo a relação entre Pclass x Fare

- Nesse momento pode surgir a pergunta do motivo de termos **pessoas pagando o mesmo preço em classes diferentes**
    - Procurando no Kaggle, [essa resposta](https://www.kaggle.com/questions-and-answers/128215) ajuda a explicar um pouco:


_"Se você conseguiu economizar e guardar todo o seu dinheiro para uma viagem para mudar sua vida e ir para a América, você é uma terceira classe, um imigrante que precisa trazer sua própria comida a bordo e ter em mente como suas provisões se sustentariam. A segunda classe era formada por trabalhadores de colarinho branco que podiam se dar ao luxo de viajar com um pouco de lazer, sem se preocupar com algo tão comum como comida. Primeira classe significa que você nasceu com dinheiro e não precisa sujar as mãos com o trabalho. Eles não eram separados apenas pelo custo, eles também eram separados por onde poderiam ir e o que recebiam."_ 

- Também tem um link do Reddit que detalha bem essa questão de classe x preço
    - https://www.reddit.com/r/AskHistorians/comments/11dlsn6/why_was_the_cost_of_tickets_on_the_titanic_so/?onetap_auto=true
    - Podemos pensar em direcionar uma **análise relacionando ao preço da cabine**, porém **essa variável possui 77.1% dos valores vazios**

<a id='Embarque'></a>
- <font color='blue'> Entendendo sobre as **colunas de embarque** </font>
    - Inicialmente pensamos não existir relação de ordem entre a coluna de embarque, mas pesquisando [nesse artigo](https://pt.wikipedia.org/wiki/RMS_Titanic) percebemos que **existe sim essa correlação**
    
    
_"O Titanic deixou o porto de **Southampton**...Em seguida ele foi para **Cherbourg-Octeville** na França...em sua maioria da primeira classe.O Titanic então rumou para **Queenstown** na Irlanda...principalmente imigrantes da terceira classe que pretendiam tentar a vida nos Estados Unidos."_ 

In [ ]:
# Podemos importar novamente as bases para "recuperar" a coluna de embarque

In [ ]:
# Como temos valores vazios, podemos novamente fazer o tratamento dos dados

- O que podemos fazer agora é colocar o porto em ordem de embarque (S depois C e depois Q)
    - Podemos fazer isso através do **[OrdinalEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html)**

In [ ]:
# Importando

In [ ]:
# Criando o encoder

In [ ]:
# Fazendo o fit com os dados

In [ ]:
# Podemos então adicionar essa coluna na base de treino original

In [ ]:
# E fazer o fit com os dados de teste

In [ ]:
# E adicionar na base de teste original

In [ ]:
# Agora podemos eliminar as colunas desnecessárias

In [ ]:
# Visualizando novamente a correlação
fig, ax = plt.subplots(figsize=(10,5))

sns.heatmap(treino.corr(), annot=True, fmt=".2f")

plt.show()

<a id='Familia'></a>
- <font color='blue'> Entendendo agora as **colunas de família** </font>
    - Podemos usar a regressão logística e a árvore de classificação para verificar a importância de cada uma dessas colunas

In [ ]:
# Separando X e y

In [ ]:
# Usando a regressão logística nos dados

In [ ]:
# Verificando a importância

In [ ]:
# Agora usando a árvore de classificação

In [ ]:
# Verificando a importância

In [ ]:
# Criando um DataFrame

In [ ]:
# Podemos apenas manter as colunas mais relevantes

<a id='modelo'></a> 
### Usando essa nova base no modelo

In [ ]:
# Visualizando a base
treino.head(3)

- Podemos selecionar os mesmos modelos que vimos anteriormente (consulte arquivo da **[parte 1](https://github.com/lucaslealx/Titanic/blob/main/Parte1.ipynb)**)
    - **Árvore de classificação**
        - https://scikit-learn.org/stable/modules/tree.html#classification
    - **Classificação dos vizinhos mais próximos**
        - https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier
    - **Regressão Logística**
        - https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression
- Antes de usar os algoritmos, precisamos separar a base de treino em **treino e validação**
    - Vamos fazer isso utilizando o **train_test_split**
        - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
# Importando o train_test_split
from sklearn.model_selection import train_test_split

In [ ]:
# Separando a base de treino em X e y
X = treino.drop(['PassengerId','Survived'],axis=1)
y = treino.Survived

In [ ]:
# Separando em treino e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)

- Para a **árvore de classificação**

In [ ]:
# Fazendo a importação
from sklearn import tree

In [ ]:
# Criando o classificador
clf_ac = tree.DecisionTreeClassifier(random_state=42)

In [ ]:
# Fazendo o fit com os dados
clf_ac = clf_ac.fit(X_train,y_train)

In [ ]:
# Fazendo a previsão
y_pred_ac = clf_ac.predict(X_val)

- Para o **KNeighborsClassifier**

In [ ]:
# Importando
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# Criando o classificador
clf_knn = KNeighborsClassifier(n_neighbors=3)

In [ ]:
# Fazendo o fit com os dados
clf_knn = clf_knn.fit(X_train,y_train)

In [ ]:
# Fazendo a previsão
y_pred_knn = clf_knn.predict(X_val)

- E para a **Regressão Logística**

In [ ]:
# Importando
from sklearn.linear_model import LogisticRegression

In [ ]:
# Criando o classificador
clf_rl = LogisticRegression(random_state=42,max_iter=1000)

In [ ]:
# Fazendo o fit com os dados
clf_rl = clf_rl.fit(X_train,y_train)

In [ ]:
# Fazendo a previsão
y_pred_rl = clf_rl.predict(X_val)

- Vamos novamente **avaliar os modelos**
    - Acurácia (método de avaliação usado na competição):
        - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html
    - Matriz de confusão (ajuda a visualizar a distribuição dos erros):
        - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html

- Avaliando a **acurácia**

In [ ]:
# Importando
from sklearn.metrics import accuracy_score

In [ ]:
# Para a árvore
accuracy_score(y_val, y_pred_ac)

In [ ]:
# Para o knn
accuracy_score(y_val, y_pred_knn)

In [ ]:
# Para a regressão logística
accuracy_score(y_val, y_pred_rl)

- Podemos salvar esses valores em um DataFrame para visualizarmos nossa evolução

In [ ]:
# Criando primeiramente o DataFrame

In [ ]:
# Adicionando novas colunas no DafaFrame

In [ ]:
# Visualizando

- Avaliando a **matriz de confusão**

In [ ]:
# Importando
from sklearn.metrics import confusion_matrix

In [ ]:
# Para a árvore
confusion_matrix(y_val, y_pred_ac)

In [ ]:
# Para o knn
confusion_matrix(y_val, y_pred_knn)

In [ ]:
# Para a regressão logística
confusion_matrix(y_val, y_pred_rl)

- Sempre que quisermos, podemos **[voltar](#ajustes)** e continuar fazendo os ajustes necessários na nossa base

### Fazendo a previsão para os dados de teste
- Vamos usar o modelo com melhor precisão para fazer o predict na base de teste

In [ ]:
# Visualizando o X_train
X_train.head(3)

In [ ]:
# Visualizando a base de teste
teste.head(3)

In [ ]:
# Para a base de teste ser igual a base de treino, precisamos eliminar a coluna de id
X_teste = teste.drop('PassengerId',axis=1)

In [ ]:
# Utilizando a regressão logística na base de teste
y_pred = clf_rl.predict(X_teste)

In [ ]:
# Criando uma nova coluna com a previsão na base de teste
teste['Survived'] = y_pred

In [ ]:
# Selecionando apenas a coluna de Id e Survived para fazer o envio
base_envio = teste[['PassengerId','Survived']]

In [ ]:
# Exportando para um csv
base_envio.to_csv('resultados5.csv',index=False)